In [ ]:
%pip install openai python-dotenv crewai crewai[tools] \
pandas numpy scipy scikit-learn matplotlib seaborn \
langchain langchain-openai langchain-community


In [ ]:
OPENROUTER_API_KEY = "sk-or..."

In [ ]:
import os
import io
import pandas as pd
from openai import OpenAI
from typing import List, Optional, Literal
from contextlib import redirect_stdout
from crewai import Agent, Task
from crewai.tools import tool

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import io

In [ ]:
# Define the possible intentions
AgentName = Literal["WRITER", "DATA_ANALYST", "RAG"]

# --- OLLAMA CLIENT SETUP ---
# Point the client to the local Ollama server running in this Colab instance
# The api_key is not required for local Ollama, but the library needs a value.
client = OpenAI(
    base_url='https://90ee-175-110-114-182.ngrok-free.app/v1',
    api_key='ollama',
)
# Define the model we pulled earlier
OLLAMA_MODEL = "qwen3:1.7b"

# --- Helper function to read file content ---
def _read_attachment_content(attachments: List[str]) -> Optional[str]:
    if not attachments:
        return None
    try:
        filepath = attachments[0]
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading attachment: {e}")
        return None

In [ ]:
# --- 1. Implemented Specialist Agents ---

class WriterAgent:
    def _build_prompt(self, query: str, context: Optional[str] = None) -> str:
        if context:
            return f"""You are a helpful writing assistant. The user's instruction is: "{query}". Apply this instruction to the following text:\n\n{context}"""
        else:
            return f"""You are a helpful writing assistant. Fulfill the user's request: "{query}"."""

    def process(self, query: str, attachments: Optional[List[str]] = None):
        print(f"✍️ [WriterAgent Activated]")
        print(f"   Task: Executing writing task...")
        content = _read_attachment_content(attachments)
        prompt = self._build_prompt(query, content)
        response = client.chat.completions.create(model=OLLAMA_MODEL, messages=[{"role": "user", "content": prompt}])
        result = response.choices[0].message.content
        print("\n--- Writer Agent Output ---\n" + result + "\n-------------------------\n")


# class DataAnalystAgent:
#     def _create_code_generation_prompt(self, query: str, df_head: str) -> str:
#         return f"""You are a Python data analysis expert. A user has a pandas DataFrame named `df`. The user's request is: "{query}".
# The DataFrame's head is:
# {df_head}
# Your task is to write a short Python script to answer the request.
# RULES:
# 1. You MUST use the `print()` function to output the final answer.
# 2. Your output MUST be only the raw Python code. Do NOT include explanations or markdown.
# Example Request: "What is the total revenue?"
# Example Output:
# print(df['Revenue'].sum())"""

#     def process(self, query: str, attachments: Optional[List[str]] = None):
#         print(f"📊 [DataAnalystAgent Activated]")
#         if not attachments or not attachments[0].endswith('.csv'):
#             print("   Error: DataAnalystAgent requires a .csv file.")
#             return

#         try:
#             df = pd.read_csv(attachments[0])
#             prompt = self._create_code_generation_prompt(query, df.head().to_string())
#             print("   Task: Generating pandas code...")
#             response = client.chat.completions.create(model=OLLAMA_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0.0)
#             generated_code = response.choices[0].message.content.strip().replace("```python", "").replace("```", "")
#             print(f"   Generated Code:\n---\n{generated_code}\n---")
#             print("   Executing code...")
#             buffer = io.StringIO()
#             with redirect_stdout(buffer):
#                 exec(generated_code, {'df': df, 'pd': pd})
#             result = buffer.getvalue()
#             print("\n--- Data Analyst Output ---\n" + result.strip() + "\n---------------------------\n")
#         except Exception as e:
#             print(f"An error occurred in DataAnalystAgent: {e}")


class RAGPoweredAgent:
    def process(self, query: str, attachments: Optional[List[str]] = None):
        print(f"🧠 [RAGPoweredAgent Activated] - (Placeholder)")
        # Actual RAG logic would go here
        print("\n--- RAG Agent Output ---\n(This is a placeholder. No real action taken.)\n------------------------\n")

In [ ]:
# --- 2. The Intention Finder Agent ---
class IntentionFinderAgent:
    """
    This agent uses a strict system prompt and robust parsing to get a
    reliable keyword from the local model.
    """
    def _create_messages(self, query: str, has_attachment: bool) -> list:
        # We KEEP the strong system prompt. This is good practice.
        system_prompt = """You are a silent, efficient routing API. Your only job is to classify a user's query into one of three categories: WRITER, DATA_ANALYST, or RAG.

Do not provide any explanation, preamble, or thought process. Your entire response MUST be a single word from the list.

The categories are:
- WRITER: For creating, summarizing, or rephrasing text.
- DATA_ANALYST: For analyzing structured data in files like CSVs.
- RAG: For answering specific questions about a document's content or for general knowledge questions.
"""
        user_prompt = f"""Query: "{query}"
Attachment Present: {has_attachment}

Category:"""

        return [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

    def find_intention(self, query: str, attachments: Optional[List[str]] = None) -> AgentName:
        messages = self._create_messages(query, bool(attachments))

        response = client.chat.completions.create(
            model=OLLAMA_MODEL,
            messages=messages,
            temperature=0.0
            # THE "stop" PARAMETER HAS BEEN REMOVED!
        )

        raw_output = response.choices[0].message.content

        # We KEEP the smart parsing logic. This is our safety net.
        intention = ""
        for keyword in ["WRITER", "DATA_ANALYST", "RAG"]:
            if keyword in raw_output.upper():
                intention = keyword
                break  # Stop after finding the first match

        if intention:
            print(f"🤖 Intention Finder determined route: -> {intention}")
            return intention
        else:
            # This will now only trigger if the model output is truly useless.
            print(f"⚠️ Warning: Could not parse a valid agent from model output ('{raw_output}'). Defaulting to RAG.")
            return "RAG"

In [ ]:
# Create dummy meeting notes file
with open("meeting_notes.txt", "w") as f:
    f.write("""Meeting Title: Q3 Strategy Review
Date: October 26, 2023
Attendees: Alice, Bob, Charlie

Discussion Points:
- Q3 sales exceeded targets by 15%, driven by the new "Innovate" product line.
- Marketing feedback suggests our social media campaigns are performing well, but email engagement is down.
- Bob presented a proposal for a new R&D project, codenamed "Project Phoenix". It focuses on AI integration and requires an initial budget of $250,000.
- Charlie raised concerns about supply chain delays impacting the "Legacy" product.

Action Items:
- Alice to draft the "Project Phoenix" budget proposal for a final review.
- Marketing team to analyze and revamp the email campaign strategy.
- Charlie to investigate alternative suppliers for the "Legacy" product components.
""")

# Create dummy sales data file
with open("sales_data.csv", "w") as f:
    f.write("""Date,Product,Category,UnitsSold,Revenue
2023-09-01,Innovate,AI,50,5000
2023-09-02,Legacy,Hardware,30,1500
2023-09-05,Innovate,AI,75,7500
2023-09-07,Innovate,AI,60,6000
2023-09-10,Legacy,Hardware,25,1250
2023-09-12,Support,Service,100,2000
2023-09-15,Innovate,AI,120,12000
2023-09-20,Legacy,Hardware,15,750
2023-09-25,Support,Service,150,3000
""")

print("✅ Libraries installed and data files created.")

✅ Libraries installed and data files created.


In [ ]:
# --- 3. Main Execution Logic ---
intention_finder = IntentionFinderAgent()
agent_map = {"WRITER": WriterAgent(), "DATA_ANALYST": DataAnalystAgent(), "RAG": RAGPoweredAgent()}

test_queries = [
    # {"query": "Please summarize the attached meeting notes.", "attachments": ["meeting_notes.txt"]},
    # {"query": "What was the total revenue in this spreadsheet?", "attachments": ["sales_data.csv"]},
    # {"query": "Which product sold the most units?", "attachments": ["sales_data.csv"]},
    {"query": "Write an email to my team about the upcoming holiday.", "attachments": None},
    {"query": "what did the report say about Q3 sales?", "attachments": ["meeting_notes.txt"]},
]

for i, test in enumerate(test_queries):
    print(f"\n" + "="*60)
    print(f"Processing Test Case #{i+1} | Query: \"{test['query']}\"")
    print("-"*60)
    intention = intention_finder.find_intention(query=test["query"], attachments=test["attachments"])
    selected_agent = agent_map.get(intention)
    if selected_agent:
        selected_agent.process(query=test["query"], attachments=test["attachments"])


Processing Test Case #1 | Query: "Write an email to my team about the upcoming holiday."
------------------------------------------------------------
🤖 Intention Finder determined route: -> WRITER
✍️ [WriterAgent Activated]
   Task: Executing writing task...


KeyboardInterrupt: 

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] ="AI"

In [ ]:
import pandas as pd

# Create a sample dataframe
data = {
    'Date': ['2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09'],
    'Product': ['Laptop', 'Mouse', 'Laptop', 'Keyboard', 'Mouse'],
    'UnitsSold': [10, 50, 12, 30, 45],
    'Price': [1200, 25, 1200, 75, 25]
}
df = pd.DataFrame(data)

# Calculate Revenue column
df['Revenue'] = df['UnitsSold'] * df['Price']

# Save to CSV
df.to_csv('sales_data.csv', index=False)

print("sales_data.csv created successfully!")
print(df.head())

sales_data.csv created successfully!
         Date   Product  UnitsSold  Price  Revenue
0  2024-01-05    Laptop         10   1200    12000
1  2024-01-06     Mouse         50     25     1250
2  2024-01-07    Laptop         12   1200    14400
3  2024-01-08  Keyboard         30     75     2250
4  2024-01-09     Mouse         45     25     1125


In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from langchain_google_genai import ChatGoogleGenerativeAI

from crewai import LLM

llm = LLM(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
)
# Initialize the Gemini 1.5 Flash model via the Generative Language API
# llm = ChatGoogleGenerativeAI(
#     model="gemini/gemini-1.5-flash",
#     temperature=0.1
# )

# --- FINAL CORRECTED TOOL DEFINITION ---
# The 'llm' and 'tool' arguments are passed directly to the tool's constructor.
# The 'config' parameter is not needed for this setup and has been removed.
data_analysis_tool = CSVSearchTool(
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini/gemini-2.0-flash",
                temperature=0.5,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    ),
    csv='sales_data.csv',
    llm=llm,
    tool='qa'
)

# --- AGENT DEFINITIONS (No changes here) ---
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Analyze the sales data from the CSV file to extract key insights, perform calculations, and identify trends.",
    backstory=(
        "You are an expert data analyst with a knack for finding hidden patterns in sales data. "
        "You are proficient in using data analysis tools to answer business questions and provide actionable insights. "
        "You focus on clarity and accuracy in your findings."
    ),
    tools=[data_analysis_tool],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

insights_reporter_agent = Agent(
    role="Insights Reporter",
    goal="Take the findings from the Data Analyst and compile them into a clear, concise, and easy-to-understand report.",
    backstory=(
        "You are a skilled communicator who specializes in translating complex data analysis into business-friendly reports. "
        "Your reports are known for their clarity, structure, and actionable recommendations. You avoid jargon and get straight to the point."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# --- TASK DEFINITIONS (No changes here)---
analysis_task = Task(
    description=(
        "Analyze the 'sales_data.csv' file. Your analysis must include the following calculations and insights:\n"
        "1. What is the total revenue for the entire period?\n"
        "2. Which product generated the most revenue and how much was it?\n"
        "3. Which product sold the most units?\n"
        "Provide only the factual results of your analysis, without any additional commentary."
    ),
    expected_output=(
        "A bullet-point list containing the precise answers to the questions. "
        "Example: '- Total Revenue: $XXXXX. - Top Product by Revenue: Product Y with $YYYY. - Top Product by Units Sold: Product Z.'"
    ),
    agent=data_analyst_agent
)

reporting_task = Task(
    description=(
        "Review the analysis provided by the Data Analyst. "
        "Format the findings into a clean, well-structured report. "
        "The report should have a clear title, a summary of the key findings, and a concluding statement."
    ),
    expected_output=(
        "A formatted report that is ready for a business stakeholder to read. "
        "It must include a title, a bullet-point list of the key insights, and a brief conclusion."
    ),
    agent=insights_reporter_agent,
    context=[analysis_task]
)

# --- CREW DEFINITION (No changes here) ---
data_analysis_crew = Crew(
    agents=[data_analyst_agent, insights_reporter_agent],
    tasks=[analysis_task, reporting_task],
    process=Process.sequential,
    verbose= True
)

# --- KICK OFF THE CREW (No changes here) ---
result = data_analysis_crew.kickoff()

# Print the final, formatted result
print("\n\n########################")
print("## Final Report")
print("########################\n")
print(result)

/usr/local/lib/python3.11/dist-packages/alembic/config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bd05841e-3972-40eb-b915-40e1c2b7345d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task: Analyze the 'sales_data.csv' file. Your analysis must include the following calculations and insights:   │
│  1. What is the total revenue for the entire period?                                                            │
│  2. Which product generated the most revenue and how much was it?                                               │
│  3. Which product sold the most units?                                                                          │
│  Provide only the factual results of your analysis, without any additional commentary.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insights Reporter                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Sales Performance Report**                                                                                   │
│                                                                                                                 │
│  **Key Insights:**                                                                                              │
│                                                                                                                 │
│  *   **Total Revenue:** The company generated a total revenue of $46,400.                                       │
│  *   **Top Product (Revenue):** Laptops were the top-performing product in terms of revenue, contributing       │
│  $26,400 to the total.                                                                                          │
│  *   **Top Product (Units Sold):** Mice were the top-selling product by volume, with 85 units sold.             │
│                                                                                                                 │
│  **Conclusion:**                                                                                                │
│                                                                                                                 │
│  The company achieved a solid revenue performance. While laptops drive the most revenue, mice lead in unit      │
│  sales, indicating a potential difference in profit margins and market penetration strategies between the two   │
│  products. Further analysis into the profitability of each product line and the effectiveness of current sales  │
│  strategies is recommended.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b2dfaff2-314a-46b2-a2ba-9a2eb9a79f08                                                                     │
│  Agent: Insights Reporter                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bd05841e-3972-40eb-b915-40e1c2b7345d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Sales Performance Report**                                                                     │
│                                                                                                                 │
│  **Key Insights:**                                                                                              │
│                                                                                                                 │
│  *   **Total Revenue:** The company generated a total revenue of $46,400.                                       │
│  *   **Top Product (Revenue):** Laptops were the top-performing product in terms of revenue, contributing       │
│  $26,400 to the total.                                                                                          │
│  *   **Top Product (Units Sold):** Mice were the top-selling product by volume, with 85 units sold.             │
│                                                                                                                 │
│  **Conclusion:**                                                                                                │
│                                                                                                                 │
│  The company achieved a solid revenue performance. While laptops drive the most revenue, mice lead in unit      │
│  sales, indicating a potential difference in profit margins and market penetration strategies between the two   │
│  products. Further analysis into the profitability of each product line and the effectiveness of current sales  │
│  strategies is recommended.                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



########################
## Final Report
########################

**Sales Performance Report**

**Key Insights:**

*   **Total Revenue:** The company generated a total revenue of $46,400.
*   **Top Product (Revenue):** Laptops were the top-performing product in terms of revenue, contributing $26,400 to the total.
*   **Top Product (Units Sold):** Mice were the top-selling product by volume, with 85 units sold.

**Conclusion:**

The company achieved a solid revenue performance. While laptops drive the most revenue, mice lead in unit sales, indicating a potential difference in profit margins and market penetration strategies between the two products. Further analysis into the profitability of each product line and the effectiveness of current sales strategies is recommended.


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

NotebookAccessError: Notebook does not have access to secret openai_api_key